In [1]:
import os
os.chdir("../delphes")

In [ ]:
import torch
from torch_geometric.data import Dataset, DataLoader
import train
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

setGPU: Setting GPU to: 0


In [3]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm[np.isnan(cm)] = 0.0

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.ylabel('True label')
    plt.xlim(-1, len(target_names))
    plt.ylim(-1, len(target_names))
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()

In [4]:
device = torch.device("cuda")

In [5]:
ds = train.DelphesDataset(".", 5000)
ds.raw_dir = "raw2"
ds.processed_dir = "processed2"
ds = torch.utils.data.Subset(ds, np.arange(start=4000, stop=5000))
d = DataLoader(ds, batch_size=1)

In [6]:
model = train.PFNet(10, 512).to(device=device)
model.load_state_dict(torch.load("model_20.pth"))
model.eval()

PFNet(
  (conv1): GATConv(10, 512, heads=4)
  (conv2): GATConv(512, 512, heads=4)
  (conv3): GATConv(512, 512, heads=4)
  (nn): Sequential(
    (0): Linear(in_features=522, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): LeakyReLU(negative_slope=0.01)
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=512, out_features=512, bias=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=512, out_features=12, bias=True)
  )
)

In [7]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print("model has {:.2E} parameters".format(params))

model has 3.19E+06 parameters


In [8]:
cms = []
cms2 = []
pred_ps = []
true_ps = []
pred_ps2 = []
true_ps2 = []
ncand_true = []
ncand_pred = []

for _d in d:
    _d = _d.to(device=device)
    
    cands_tower_id, cands_trk_id, cands_tower_p, cands_trk_p = model(_d)
    
    cands_tower_p[:, 0] = torch.exp(cands_tower_p[:, 0])
    cands_trk_p[:, 0] = torch.exp(cands_trk_p[:, 0])
    
    pred_ids_tower = model.decode_ids(cands_tower_id)
    true_ids_tower = _d.y_tower[:, 0]
    
    pred_ids_trk = model.decode_ids(cands_trk_id)
    true_ids_trk = _d.y_trk[:, 0]
    
#     y_id_pred = torch.argmax(_pred_id, axis=-1)
#     _pred_p[y_id_pred!=0, 0] = torch.exp(_pred_p[y_id_pred!=0, 0])

    cm = confusion_matrix(true_ids_trk.flatten().cpu(), pred_ids_trk.flatten().detach().cpu(),
        labels=range(len(train.map_candid_to_numid)))
    cm2 = confusion_matrix(true_ids_tower.flatten().cpu(), pred_ids_tower.flatten().detach().cpu(),
        labels=range(len(train.map_candid_to_numid)))
    
    msk = (pred_ids_tower!=0) & (true_ids_tower!=0)
    pred_ps += cands_tower_p[msk].detach().cpu()
    true_ps += _d.y_tower[msk][:, 1:].detach().cpu()
    
    msk = (pred_ids_trk!=0) & (true_ids_trk!=0)
    pred_ps2 += cands_trk_p[msk].detach().cpu()
    true_ps2 += _d.y_trk[msk][:, 1:].detach().cpu()
    
    cms += [cm]
    cms2 += [cm2]
    
    ncand_true += [int((true_ids_tower!=0).sum() + (true_ids_trk!=0).sum())]
    ncand_pred += [int((pred_ids_tower!=0).sum() + (pred_ids_trk!=0).sum())]
cm = sum(cms)
cm2 = sum(cms2)
pred_p = np.stack(pred_ps)
true_p = np.stack(true_ps)
pred_p2 = np.stack(pred_ps2)
true_p2 = np.stack(true_ps2)

RuntimeError: CUDA error: no kernel image is available for execution on the device

In [ ]:
import networkx as nx
import torch_geometric

In [ ]:
adj_matrix = torch_geometric.utils.to_dense_adj(_d.edge_index).cpu().numpy()[0, :, :]
x = _d.x.cpu().numpy()
# y_p = _d.y_tower.cpu().numpy()
# y_p_pred = _pred_p[(y_id_pred!=0).cpu().numpy(), :].detach().cpu().numpy()

# y_id = _d.y_id.cpu().numpy()

colors = {
    0: "r",
    1: "b",
}

g = nx.from_numpy_matrix(adj_matrix)
for i in range(len(x)):
    g.nodes[i]["color"] = colors[x[i, 0]]
    g.nodes[i]["s"] = np.abs(x[i, 4] + x[i, 3])
    if x[i, 0] == 0:
        g.nodes[i]["eta"] = x[i, 1]
        g.nodes[i]["phi"] = x[i, 2]
    elif x[i, 0] == 1:
        g.nodes[i]["eta"] = x[i, 5]
        g.nodes[i]["phi"] = x[i, 6]
        
pos = {i: (g.nodes[i]["eta"], g.nodes[i]["phi"]) for i in range(len(g))}

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(
    x[:, 1],
    x[:, 2],
    s=np.abs(x[:, 4] + x[:, 3]),
    label="det")

m = _d.y_tower[:, 0] != 0
plt.scatter(
    _d.y_tower[m][:, 2].cpu(),
   _d.y_tower[m][:, 3].cpu(),
    s=_d.y_tower[m][:, 1].cpu(),
    label="gen")

m = _d.y_trk[:, 0] != 0
plt.scatter(
    _d.y_trk[m][:, 2].cpu(),
   _d.y_trk[m][:, 3].cpu(),
    s=_d.y_trk[m][:, 1].cpu(),
    label="gen-trk")

plt.legend()

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.title("input graph")
edges = np.array(list(g.edges))
edges = list(edges[np.random.permutation(len(edges))][:500])

nx.draw_networkx(g, pos,
    node_size=[g.nodes[n]["s"] for n in g.nodes],
    node_color=[g.nodes[n]["color"] for n in g.nodes],
    with_labels=False, alpha=0.2, edgelist=edges)
plt.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xlim(-6, 6)
plt.ylim(-4, 4)

In [ ]:
plt.figure(figsize=(3*4, 4))

ax = plt.subplot(1,3,1)
plt.title("input graph")
nx.draw_networkx(g, pos,
    node_size=[g.nodes[n]["s"] for n in g.nodes],
    node_color=[g.nodes[n]["color"] for n in g.nodes],
    with_labels=False, alpha=0.2, ax=ax)
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xlim(-6, 6)
plt.ylim(-4, 4)

plt.subplot(1,3,2)
m = _d.y_tower[:, 0] != 0
plt.scatter(
    _d.y_tower[m][:, 2].cpu(),
   _d.y_tower[m][:, 3].cpu(),
    s=_d.y_tower[m][:, 1].cpu(),
    label="gen-tower")

m = pred_ids_tower != 0
plt.scatter(
    cands_tower_p[m][:, 1].detach().cpu(),
    cands_tower_p[m][:, 2].detach().cpu(),
    s=cands_tower_p[m][:, 0].detach().cpu(), alpha=0.5, label="pred")
plt.xlim(-6, 6)
plt.ylim(-4, 4)
plt.tight_layout()
plt.legend(loc=1)

plt.subplot(1,3,3)
m = _d.y_trk[:, 0] != 0
plt.scatter(
    _d.y_trk[m][:, 2].cpu(),
    _d.y_trk[m][:, 3].cpu(),
    s=_d.y_trk[m][:, 1].cpu(),
    label="gen-trk")

m = pred_ids_trk != 0
plt.scatter(
    cands_trk_p[m][:, 1].detach().cpu(),
    cands_trk_p[m][:, 2].detach().cpu(),
    s=cands_trk_p[m][:, 0].detach().cpu(), alpha=0.5, label="pred")
plt.xlim(-6, 6)
plt.ylim(-4, 4)
plt.tight_layout()
plt.legend(loc=1)

plt.savefig("test.pdf")

In [ ]:
target_names = ["none", "211", "-211", "130", "22", "11", "-11", "13", "-13"]

In [ ]:
plot_confusion_matrix(cm, target_names, normalize=False)
plt.xlim(-0.5, cm.shape[0]-0.5)
plt.ylim(-0.5, cm.shape[0]-0.5)

In [ ]:
plot_confusion_matrix(cm2, target_names, normalize=False)
plt.xlim(-0.5, cm.shape[0]-0.5)
plt.ylim(-0.5, cm.shape[0]-0.5)

In [ ]:
plt.figure(figsize=(5,5))
plt.xlim(0, 3000)
plt.ylim(0, 3000)
plt.plot([0,5000],[0,5000], color="black")
plt.scatter(ncand_true, ncand_pred, marker=".")

In [ ]:
plt.figure(figsize=(6, 4))
b = np.linspace(0, 10, 100)
plt.hist(true_p[:, 0], bins=b, histtype="step", lw=2);
plt.hist(pred_p[:, 0], bins=b, histtype="step", lw=2);
plt.yscale("log")
#plt.ylim(100,1e4)

In [ ]:
plt.figure(figsize=(6, 4))
b = np.linspace(0, 10, 100)
plt.hist(true_p2[:, 0], bins=b, histtype="step", lw=2);
plt.hist(pred_p2[:, 0], bins=b, histtype="step", lw=2);
plt.yscale("log")
#plt.ylim(100,1e4)

In [ ]:
plt.figure(figsize=(6, 4))
b = np.linspace(0, 100, 100)
plt.hist(true_p[:, 0], bins=b, histtype="step", lw=2);
plt.hist(pred_p[:, 0], bins=b, histtype="step", lw=2);
plt.yscale("log")

In [ ]:
plt.figure(figsize=(6, 4))
b = np.linspace(0, 100, 100)
plt.hist(true_p2[:, 0], bins=b, histtype="step", lw=2);
plt.hist(pred_p2[:, 0], bins=b, histtype="step", lw=2);
plt.yscale("log")

In [ ]:
plt.figure(figsize=(6, 4))
b = np.linspace(-6, 6, 100)
plt.hist(true_p[:, 1], bins=b, histtype="step", lw=2);
plt.hist(pred_p[:, 1], bins=b, histtype="step", lw=2);

In [ ]:
plt.figure(figsize=(6, 4))
b = np.linspace(-6, 6, 100)
plt.hist(true_p2[:, 1], bins=b, histtype="step", lw=2);
plt.hist(pred_p2[:, 1], bins=b, histtype="step", lw=2);

In [ ]:
plt.figure(figsize=(6, 4))
b = np.linspace(-4, 4, 100)
plt.hist(true_p[:, 2], bins=b, histtype="step", lw=2);
plt.hist(pred_p[:, 2], bins=b, histtype="step", lw=2);

In [ ]:
plt.figure(figsize=(6, 4))
b = np.linspace(-4, 4, 100)
plt.hist(true_p2[:, 2], bins=b, histtype="step", lw=2);
plt.hist(pred_p2[:, 2], bins=b, histtype="step", lw=2);

In [ ]:
rp = np.random.permutation(len(pred_p))[:5000]
rp2 = np.random.permutation(len(pred_p2))[:5000]

In [ ]:
np.corrcoef(true_p[:, 0], pred_p[:, 0])[0,1]

In [ ]:
np.corrcoef(true_p2[:, 0], pred_p2[:, 0])[0,1]

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(true_p[rp, 0], pred_p[rp, 0], marker=".", alpha=0.5)
plt.xlim(0, 10)
plt.ylim(0, 10)
plt.plot([0, 10], [0, 10], color="black")

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(true_p[rp, 0], pred_p[rp, 0], marker=".", alpha=0.5)
plt.xlim(0.1, 1000)
plt.ylim(0.1, 1000)
plt.plot([0.1, 1000], [0.1, 1000], color="black")
plt.xscale("log")
plt.yscale("log")

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(true_p2[rp2, 0], pred_p2[rp2, 0], marker=".", alpha=0.5)
plt.xlim(0.1, 1000)
plt.ylim(0.1, 1000)
plt.plot([0.1, 1000], [0.1, 1000], color="black")
plt.xscale("log")
plt.yscale("log")

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(true_p[rp, 1], pred_p[rp, 1], marker=".", alpha=0.5)
plt.plot([-7, 7], [-7, 7], color="black")
plt.xlim(-7,7)
plt.ylim(-7,7)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(true_p[rp, 2], pred_p[rp, 2], marker=".", alpha=0.5)
plt.plot([-4, 4], [-4, 4], color="black")
plt.xlim(-4,4)
plt.ylim(-4,4)

In [ ]:
msk = _d.x[:, 0] == 1
plt.figure(figsize=(4,4))
plt.scatter(_d.x[msk, 1].cpu(), _d.y_trk[:, 2].cpu(), marker=".")

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(_d.x[msk, 2].cpu(), _d.y_trk[:, 3].cpu(), marker=".")

In [ ]:
msk = _d.x[:, 0] == 0
plt.figure(figsize=(4,4))
plt.scatter(_d.x[msk, 1].cpu(), _d.y_tower[:, 2].cpu(), marker=".")

In [ ]:
msk = _d.x[:, 0] == 0
plt.figure(figsize=(4,4))
plt.scatter(_d.x[msk, 2].cpu(), _d.y_tower[:, 3].cpu(), marker=".")

In [ ]:
msk = _d.x[:, 0] == 0
plt.figure(figsize=(4,4))
plt.scatter(_d.x[msk, 3].cpu() + _d.x[msk, 4].cpu(), _d.y_tower[:, 1].cpu(), marker=".")
plt.xscale("log")
plt.yscale("log")

In [ ]:
msk = _d.x[:, 0] == 1
plt.figure(figsize=(4,4))
plt.scatter(_d.x[msk, 4].cpu(), _d.y_trk[:, 1].cpu(), marker=".")
plt.xscale("log")
plt.yscale("log")